In [140]:
import pandas as pd
import numpy as np

In [141]:
df = pd.read_csv("file.csv", sep=';', encoding='latin1', na_values='idk', decimal=',')

In [142]:
df.head()

,Argus Drug Code,Product Name,ICRC Name,TradeName,Category (0-5),Category (new),TA,Responsible GSL,TA Coordinator,DLP,...,Planning Outsource,outsourced to,score,Time,Complexity,# of Topics,Complexity New,Pages,Indications,Products
0,BAY39-6251,AZELAIC ACID,0,Skinoren,2,B,GM/DI,Bianca Gagliotti,Asad Kazi,02.01.2017,...,yes,TCS,1,93.0,III,10,III,131.0,3.0,1.0
1,BAY1841788,Darolutamide (previous: ODM-201 (AR Antagonist)),ODM-201,NaN,1,A,ONC Non-TKI,Ping Ji,Susanne Kippes,04.01.2017,...,yes,ICRC-Weyer,1,124.5,III,6,IV,349.0,NaN,1.0
2,BAYA2502,NIFURTIMOX,0,Lampit,2,B,PC/Pul,Yuhong Wang,Kathrin Mastrangelo,07.01.2017,...,yes,TCS,1,98.0,III,2,V,147.0,1.0,1.0
3,BAYA2502,NIFURTIMOX,0,Lampit,2,B,PC/Pul,Yuhong Wang,Kathrin Mastrangelo,07.01.2017,...,yes,TCS,1,106.0,III,8,III,56.0,2.0,1.0
4,BAY1217389,"MPS1 INHIBIOTR ""FU"" (imidazopyr)",0,NaN,1,A,ONC Non-TKI,Cinara McCarthy,Susanne Kippes,20.01.2017,...,yes,TCS,1,54.0,III,1,V,272.0,1.0,1.0


In [143]:
y = df['Time']

In [144]:
X = df.loc[:, df.columns!='Time']


In [145]:
# Checking Pearson
df[df.columns[1:]].corr()['Time']

Category (0-5)                                          0.278319
Due Date \n(if no exact date mid of quarter planned)         NaN
score                                                        NaN
Time                                                    1.000000
# of Topics                                             0.434516
Pages                                                   0.012926
Indications                                            -0.046333
Products                                                0.174516
Name: Time, dtype: float64

In [146]:
df = df[['# of Topics', 'Products','TA']]

In [178]:
df.columns = ["topics", "products", "ta"]

In [180]:
y

0      93.0
1     124.5
2      98.0
3     106.0
4      54.0
5     223.0
6      96.5
7      71.0
8     135.0
9      41.0
10     98.0
11    177.0
12     87.0
13     95.0
14     88.0
15     95.0
16     99.0
17    103.0
18     69.0
19    119.0
20     56.0
21    165.0
22     91.0
23    104.5
24    106.0
25     41.0
26     47.0
27    155.0
28    110.0
29     76.5
30    226.0
31    174.0
32    104.5
33     91.0
Name: Time, dtype: float64

In [107]:
# Data cleaning
# Convert outsourced to to numeric
from sklearn import preprocessing as pre
# le = pre.LabelEncoder()
# le.fit(df['outsourced to'])
# df['outsourced to'] = le.transform(df['outsourced to'])

In [147]:
le = pre.LabelEncoder()
le.fit(df["TA"])
df['TA'] = le.transform(df['TA'])


In [108]:
#df = pd.get_dummies(df)

In [111]:
# pages: impute nans with mean
# from sklearn.preprocessing import Imputer
# imp = Imputer()
# imp.fit(df['Pages'].reshape(-1, 1))
# new = imp.transform(df['Pages'].reshape(-1,1))
# df['Pages'] = new

In [150]:
# Topics is fine
# Indications: Replace idk & NaNs with 1
df.Indications = df.Indications.fillna(value=1, )

AttributeError: 'DataFrame' object has no attribute 'Indications'

In [151]:
df.Products = df.Products.fillna(value=1)


In [154]:
X = df.copy()

,# of Topics,Products,TA
0,10,1.0,0
1,6,1.0,1
2,2,1.0,3
3,8,1.0,3
4,1,1.0,1
5,0,1.0,0
6,4,16.0,3
7,3,1.0,0
8,7,1.0,0
9,2,1.0,0


In [155]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [156]:
from sklearn.ensemble import RandomForestRegressor
rf  = RandomForestRegressor()
rf.fit(X, y)
print('Features by their score')
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), list(X.columns)), reverse=True))
#rf.feature_importances_

Features by their score
[(0.65959999999999996, '# of Topics'), (0.20219999999999999, 'Products'), (0.13819999999999999, 'TA')]


In [157]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [166]:
# Try linear regression
from sklearn.linear_model import LinearRegression as lr
my_lr = lr(normalize=True)
my_lr.fit(X_train_std, y_train)
my_lr.predict(X_test_std)
my_lr.score(X=X_test_std, y=y_test)

my_lr.predict(X=X_test_std)
my_lr.coef_

array([ 14.98127084,   2.48755799, -10.58347803])

In [163]:
X_test_std
a = [3.5, 2, -2]
np.asarray(a)
my_lr.predict(X=a)

C:\Users\hagen\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 186.85429775])

In [138]:
# adaboost
from sklearn.ensemble import AdaBoostRegressor
my_adaboost = AdaBoostRegressor(n_estimators=50, learning_rate=0.5, loss="linear", )
my_adaboost.fit(X_train_std, y_train)
my_adaboost.score(X=X_test_std, y=y_test)


0.20951502090821505

In [137]:
y_test

11    177.0
20     56.0
25     41.0
17    103.0
2      98.0
26     47.0
31    174.0
Name: Time, dtype: float64

In [139]:
from sklearn.svm import SVR 
my_svr  = SVR(C=1, epsilon=0.2, kernel="linear")
my_svr.fit(X_train_std, y_train)
my_svr.score(X=X_test_std, y=y_test)

-0.004653624922354771

In [41]:
#df = df[['outsourced to', '# of Topics', 'Indications', 'Products', 'TA']]


In [42]:
# from sklearn.neural_network import MLPRegressor
# my_net = MLPRegressor(hidden_layer_sizes=(3,3,1), activation='relu', alpha=0.0001, max_iter=1000)
# my_net.fit(X_train_std, y_train)
# my_net.score(X=X_test_std, y=y_test)

In [21]:
# from sklearn.model_selection import GridSearchCV
# alpha = 10.0 ** -np.arange(1,7)
# iterations = [500, 1000, 5000]
# params = {
#             'hidden_layer_sizes' : ((1,1,1), (2,2,2), (4,5,1), (3,3,1)),
#             'alpha' : alpha, 
#             'activation' : ('relu', 'logistic'), 
#             'solver' : ('adam', 'sgd'),
#             'max_iter': iterations
#          }
# my_clf = GridSearchCV(my_net, params)
# my_clf.fit(X_train_std, y_train)
# my_clf.score(X=X_test_std, y=y_test)

In [51]:
my_clf.best_estimator_

MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 3, 1), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)